In [9]:
# loading the gensim model into spacy model didn't work (couldn't convert the gensim model) 

# import spacy
# # Load the spacy model that you have installed
# nlp = spacy.load('./word2vec_model_one_gesim.model/')
# # process a sentence using the model
# doc = nlp("This is some text that I am processing with Spacy")
# # It's that simple - all of the vectors and words are assigned after this point
# # Get the vector for 'text':
# doc[3].vector
# # Get the mean vector for the entire sentence (useful for sentence classification etc.)
# doc.vector

In [10]:
import gensim
import pandas as pd
import numpy as np
from scipy import spatial

In [11]:
def load_model(name:str, path:str =""):
    return gensim.models.Word2Vec.load(path + name)
# model = gensim.models.Word2Vec.load("word2vec_model_one_gesim.model")

In [12]:
def read_csv(name: str, path:str=""):
    return pd.read_csv()

In [18]:
def read_question_data(data):
    question1_data = data["question1"].values.tolist()
    question2_data = data["question2"].values.tolist()
    labels = data["is_duplicate"].values.tolist()

    return [question1_data, question2_data, labels] 

In [14]:
# preprocessing the data (tokenizing and removing punctutation)
def gensim_preprocess(question_data):
    return [gensim.utils.simple_preprocess(str(sentence)) for sentence in question_data] 

In [15]:
def string_preprocess(question_data):
    stop_words=['the', 'a', 'an', 'and', 'is', 'be', 'will']
    new_data = []
    
    for question in question_data:
        processed_question = []
        question = str(question)
        
        #split the question to words (doesn't include stop words)
        words = [word.lower() for word in question.split() if word not in stop_words]
        for word in words:
            # to remove punctutation from string (imported from string library)
            word = word.translate(str.maketrans('', '', string.punctuation))
            word = word.replace('“', '').replace('”', '')

            if len(word) > 0:
                processed_question.append(word)
                
        new_data.append(processed_question)
    
    return new_data

In [16]:
model_gensim = load_model("word2vec_model_gensim_100.model")
model_manual = load_model("word2vec_model_manual_100.model")

In [19]:
data = read_csv

TypeError: parser_f() missing 1 required positional argument: 'filepath_or_buffer'

In [ ]:
# question1_data = df["question1"].values.tolist()
# question2_data = df["question2"].values.tolist()
# label = df["is_duplicate"].values.tolist()
# print(question1_data[0:4])

In [ ]:
# cleaned_question1_data = [gensim.utils.simple_preprocess(str(sentence)) for sentence in question1_data] 
# cleaned_question2_data = [gensim.utils.simple_preprocess(str(sentence)) for sentence in question2_data] 

In [31]:
# print(cleaned_question1_data[0:4])
# print(cleaned_question2_data[0:4])

[['what', 'is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india'], ['what', 'is', 'the', 'story', 'of', 'kohinoor', 'koh', 'noor', 'diamond'], ['how', 'can', 'increase', 'the', 'speed', 'of', 'my', 'internet', 'connection', 'while', 'using', 'vpn'], ['why', 'am', 'mentally', 'very', 'lonely', 'how', 'can', 'solve', 'it']]
[['what', 'is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market'], ['what', 'would', 'happen', 'if', 'the', 'indian', 'government', 'stole', 'the', 'kohinoor', 'koh', 'noor', 'diamond', 'back'], ['how', 'can', 'internet', 'speed', 'be', 'increased', 'by', 'hacking', 'through', 'dns'], ['find', 'the', 'remainder', 'when', 'math', 'math', 'is', 'divided', 'by']]


In [51]:
question1_word_vectors = []
count = 1

for word_list in cleaned_question1_data:
    temp_list = []
    for word in word_list:
        try:
            temp_list.append(model.wv[word])
        except:
            count += 1 
    question1_word_vectors.append(temp_list)
print(count)

28840


In [52]:
question2_word_vectors = []
count = 1
labels = []
for word_list in cleaned_question2_data:
    temp_list = []
    for word in word_list:
        try:
            temp_list.append(model.wv[word])
        except:
            count += 1 
    question2_word_vectors.append(temp_list)
print(count)

23361


In [53]:
print(len(question1_word_vectors))
print(len(question2_word_vectors))

404290
404290


In [69]:
question1_sentence_vectors = [sum(sentence) for sentence in question1_word_vectors if len(sentence)>0]
question2_sentence_vectors = [sum(sentence) for sentence in question2_word_vectors if len(sentence)>0]

In [70]:
training_data = []
for i in range(len(question1_sentence_vectors)):
    training_data.append(1 - spatial.distance.cosine(question1_sentence_vectors[i], question2_sentence_vectors[i]))

In [72]:
print(training_data[0:20])

[0.973378598690033, 0.3867042362689972, 0.7577860355377197, 0.2219158113002777, 0.8361110687255859, 0.9349866509437561, 0.08878669887781143, 0.669806182384491, 0.9755182266235352, 0.6018723845481873, 0.37011805176734924, 0.8813337683677673, 0.8732406497001648, 0.9487687945365906, 0.9977368116378784, 0.8526891469955444, 0.8565733432769775, 0.7768007516860962, 0.8304876685142517, 0.9592839479446411]


In [73]:
print(label[0:20])

[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0]


In [86]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(np.array(training_data).reshape(-1,1), np.array(label[:404260]))

In [78]:
model.wv.most_similar(model.wv["what"])

[('what', 1.0000001192092896),
 ('which', 0.5997904539108276),
 ('the', 0.4692210257053375),
 ('some', 0.35175949335098267),
 ('globalization', 0.3471813499927521),
 ('practices', 0.34091559052467346),
 ('crusades', 0.33653372526168823),
 ('horror', 0.33569419384002686),
 ('firing', 0.3314772844314575),
 ('psychological', 0.3312871754169464)]

In [114]:
df_test = pd.read_csv('test.csv')
df_test.head(5)

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [115]:
question1_test_data = df_test["question1"].values.tolist()
question2_test_data = df_test["question2"].values.tolist()

In [116]:
print(len(question1_test_data))

2345796


In [93]:
cleaned_question1_test_data = [gensim.utils.simple_preprocess(str(sentence)) for sentence in question1_test_data] 
cleaned_question2_test_data = [gensim.utils.simple_preprocess(str(sentence)) for sentence in question2_test_data] 

In [94]:
question1_word_test_vectors = []
count = 1

for word_list in cleaned_question1_test_data:
    temp_list = []
    for word in word_list:
        try:
            temp_list.append(model.wv[word])
        except:
            count += 1 
    question1_word_test_vectors.append(temp_list)
print(count)

question2_word_test_vectors = []
count = 1
labels = []
for word_list in cleaned_question1_test_data:
    temp_list = []
    for word in word_list:
        try:
            temp_list.append(model.wv[word])
        except:
            count += 1 
    question2_word_test_vectors.append(temp_list)
print(count)

425992
425992


In [105]:
question1_test_sentence_vectors = [sum(sentence) for sentence in question1_word_test_vectors if len(sentence)>0]
question2_test_sentence_vectors = [sum(sentence) for sentence in question2_word_test_vectors if len(sentence)>0]

In [106]:
test_data = []
for i in range(len(question1_test_sentence_vectors)):
    test_data.append(1 - spatial.distance.cosine(question1_test_sentence_vectors[i], question2_test_sentence_vectors[i]))

In [107]:
print(len(test_data))

3563108


In [108]:
clf.predict(np.array(test_data).reshape(-1,1))

array([0, 0, 0, ..., 0, 0, 0])